In [1]:
from langchain_community.document_loaders import PyPDFLoader

In [3]:
FILE_PATH=r"C:\Users\Anilchoudary R\Gitrepos\langchain_playground\data\CV_Anilchoudary_Rugaramji_English.pdf"

In [5]:
loader = PyPDFLoader(FILE_PATH)
loader.load()

[Document(metadata={'producer': 'Canva', 'creator': 'Canva', 'creationdate': '2025-06-14T11:41:12+00:00', 'title': 'Blue minimalist business Resume CV', 'moddate': '2025-06-14T11:41:11+00:00', 'keywords': 'DAGpOoJe7Ig,BAGpOVjcSGo,0', 'author': 'Anilchoudary', 'source': 'C:\\Users\\Anilchoudary R\\Gitrepos\\langchain_playground\\data\\CV_Anilchoudary_Rugaramji_English.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='Feature Development for SAP - DMS Adapter Product\nDeveloped and enhanced backend features using Go and frontend components with Angular for a connector product\nthat enables SAP ERP customers to integrate seamlessly with d.velop documents.\nImplemented unit, integration, and API tests to increase code coverage and reduce production defects.\nProvided ongoing product support by resolving support tickets and collaborating with QA and support teams to\nmaintain product quality.\nActively participated in Agile/Scrum ceremonies, contributing to sprint plannin

In [10]:
len(loader.load())

2

In [13]:
pages = loader.load()

In [19]:
## Chunking
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, # hyper parameter
    chunk_overlap=50 # hyper parameter
)

In [21]:
splitted_docs = splitter.split_documents(pages)

In [23]:
len(splitted_docs)

13

In [26]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [27]:
# vector DB
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

index=faiss.IndexFlatL2(384)

vector_store=FAISS(
    index=index, 
    embedding_function=embeddings,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}

)

In [28]:
vector_store.add_documents(splitted_docs)

['415857ae-baaa-4c8a-b984-f118910a8229',
 '77cc21e3-1e89-4752-8877-1fc34e6a6829',
 'fe5ad10c-1b1c-4e43-82bc-c7799829b195',
 '1b609423-431a-467c-b593-c6fc386f2d25',
 '0150e471-e742-44f1-9b70-cd4272aa0d0b',
 'befd2363-3b64-4304-8774-be5cc382f1f8',
 'ec16be87-b780-4fa0-b70e-28e6527ea854',
 'dd956ecf-f39b-42ae-bf7e-cb07d2885d50',
 'e89ebcee-579b-4024-b947-cefb31d641fc',
 '69a65c03-0741-48ef-b110-edf7efc384a8',
 '3cac45ed-253e-42fa-8174-497b2af737b7',
 '4617e868-e7a5-4b9a-bb88-55738f772e2d',
 '8bae0467-6417-4b68-9771-729ef55af923']

In [36]:
retriver = vector_store.as_retriever(
    search_kwargs={"k":10}
)

In [37]:
retriver.invoke("What are roles and responsibilites at d.velop AG")

[Document(id='415857ae-baaa-4c8a-b984-f118910a8229', metadata={'producer': 'Canva', 'creator': 'Canva', 'creationdate': '2025-06-14T11:41:12+00:00', 'title': 'Blue minimalist business Resume CV', 'moddate': '2025-06-14T11:41:11+00:00', 'keywords': 'DAGpOoJe7Ig,BAGpOVjcSGo,0', 'author': 'Anilchoudary', 'source': 'C:\\Users\\Anilchoudary R\\Gitrepos\\langchain_playground\\data\\CV_Anilchoudary_Rugaramji_English.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='Feature Development for SAP - DMS Adapter Product\nDeveloped and enhanced backend features using Go and frontend components with Angular for a connector product\nthat enables SAP ERP customers to integrate seamlessly with d.velop documents.\nImplemented unit, integration, and API tests to increase code coverage and reduce production defects.\nProvided ongoing product support by resolving support tickets and collaborating with QA and support teams to\nmaintain product quality.'),
 Document(id='ec16be87-b780-4fa0-b

In [38]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [39]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

In [40]:
import pprint

In [41]:
pprint.pprint(prompt.messages)

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


In [42]:
## context -- retriver#
## prompt -- hub
## model -- google
## parser -- langchain

In [43]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough





In [44]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [52]:
rag_chain = (
    {"context": retriver | format_docs, "question": RunnablePassthrough()} 
    | prompt 
    | model
    | StrOutputParser()
)

In [53]:
rag_chain.invoke("What is Anil's work experience in different companies?")

"Anil worked as a Junior Software Developer from May 2022 to October 2022, then as a Software Development Engineer from October 2023 to present.  He also held a Territory Executive position from June 2016 to June 2017 and completed a Master's Thesis from April 2021 to November 2021."